<a href="https://colab.research.google.com/github/akshayjain777/AI/blob/master/ResNet_practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F ## activation function
import torchvision ##contains dataset
import pdb

In [0]:
class baseBlock_layer_1_3(torch.nn.Module):
  expansion=1
  def __init__(self,input_planes,planes,stride=1,dim_change=None):
    super(baseBlock_layer_1_3, self).__init__()
    #declare convolution layers with batch norms
    self.conv1 = torch.nn.Conv2d(input_planes,planes,stride=stride,kernel_size=3,padding=1)
    self.bn1= torch.nn.BatchNorm2d(planes)
    self.conv2 = torch.nn.Conv2d(planes,planes,stride=1,kernel_size=3,padding=1)
    self.bn2=torch.nn.BatchNorm2d(planes)
    self.dim_change=dim_change
  def forward(self,x):
    #save the residue
    res = x
    output=F.relu(self.bn1(self.conv1(x)))
    output=F.relu(self.bn2(self.conv2(output)))

    if self.dim_change is not None:
      res=self.dim_change(res)

    output += res
    output= F.relu(output)

    return output

In [0]:
class baseBlock_layer_2(torch.nn.Module):
  expansion=1
  def __init__(self,input_planes,planes,stride=1,dim_change=None):
    super(baseBlock_layer_2, self).__init__()
    #declare convolution layers with batch norms
    self.conv1 = torch.nn.Conv2d(input_planes,planes,stride=stride,kernel_size=3,padding=1)
    self.pool1 = torch.nn.MaxPool2d(2,2) ## 
    self.bn1= torch.nn.BatchNorm2d(planes)
    self.dim_change=dim_change
  def forward(self,x):
    #save the residue
    res = x
    output = self.pool1(F.relu(self.bn1(self.conv1(x))))


    if self.dim_change is not None:
      res=self.dim_change(res)

    output += res
    output= F.relu(output)

    return output

In [0]:
class ResNet(torch.nn.Module):
  def __init__(self,block,num_layers,classes=10):
    super(ResNet,self).__init__()
    #according to research paper
    self.input_planes=64
    self.conv1= torch.nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1)
    self.bn1= torch.nn.BatchNorm2d(64)

    self.layer1 = self._layer(block,128,num_layers[1],stride=2)
    self.layer2= self._layer(baseBlock_layer_2,256,num_layers[2])
    self.layer3 = self._layer(block,512,num_layers[3],stride=2)
    #self.softmax = nn.Softmax(512*block.expansion)

  def _layer(self,block,planes,num_layers,stride=1):
    dim_change = None
    if stride!=1 or planes !=self.input_planes*block.expansion:
      dim_change = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes,planes*block.expansion,kernel_size=1,stride=stride),
                                       torch.nn.BatchNorm2d(planes*block.expansion))
    
    netLayers= []
    netLayers.append(block(self.input_planes,planes,stride=stride,dim_change=dim_change))
    self.input_palnes= planes * block.expansion
    for i in range(1,num_layers):
      netLayers.append(block(self.input_planes,planes))
      self.input_planes= planes * block.expansion
    
    return torch.nn.Sequential(*netLayers)
  
  def forward(self,x):
    x= F.relu(self.bn1(self.conv1(x)))
   # x= F.relu(self.bn1(self.conv1(x)))
    print(x.shape)
    print()
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)

    x=F.max_pool2d(x,4)
    x=x.view(x.size(0),-1)
    x=self.fc(x)
    x= F.log_softmax(x,dim=-1)
    #x=self.softmax(x)


    return x
  

In [0]:
def ResNet18():
    return ResNet(baseBlock_layer_1_3, [2, 2, 2, 2])

In [0]:
def test():
    net = ResNet18()
    print(torch.randn(3, 3, 32, 32).shape)
    y = net(torch.randn(3, 3, 32, 32))
    print(y.size())

In [34]:
test()

torch.Size([3, 3, 32, 32])


RuntimeError: ignored